In [1]:
import omnipath
import pandas as pd
import liana

/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
new_Omnipath_database = pd.read_csv("/data/LR_database/complexes.csv")

In [15]:
new_Omnipath_database[~new_Omnipath_database["complex"].isnull()]["member"].unique().shape

(8022,)

In [16]:
omnipath.__version__

'1.0.6'

In [97]:
#import validated interactions 
resource = omnipath.interactions.OmniPath.get()
categories = omnipath.requests.Intercell.get()

In [100]:
omnipath.requests.Complexes.get()

3.92MB [00:00, 10.0MB/s]


,name,components,components_genesymbols,stoichiometry,sources,references,identifiers,references_stripped,n_references,n_sources,n_primary_sources
0,NFY,P23511_P25208_Q13952,NFYA_NFYB_NFYC,1:1:1,CORUM;Compleat;ComplexPortal;PDB;SIGNOR;SPIKE;...,9372932;14755292;15243141,CORUM:4478;Compleat:HC1449;PDB:4awl;PDB:6qmp;P...,1;2,2,8,8
1,mTORC2,P42345_P68104_P85299_Q6R327_Q8TB45_Q9BVC4,DEPTOR_EEF1A1_MLST8_MTOR_PRR5_RICTOR,0:0:0:0:0:0,SIGNOR,None,SIGNOR:SIGNOR-C2,None,None,1,1
2,mTORC1,P42345_Q8N122_Q8TB45_Q96B36_Q9BVC4,AKT1S1_DEPTOR_MLST8_MTOR_RPTOR,0:0:0:0:0,SIGNOR,None,SIGNOR:SIGNOR-C3,None,None,1,1
3,SCF-betaTRCP,P63208_Q13616_Q9Y297,BTRC_CUL1_SKP1,1:1:1,CORUM;Compleat;SIGNOR;SPIKE,9990852,CORUM:227;Compleat:HC757;SIGNOR:SIGNOR-C5,2,1,4,4
4,CBP/p300,Q09472_Q92793,CREBBP_EP300,0:0,SIGNOR,None,SIGNOR:SIGNOR-C6,None,None,1,1
...,...,...,...,...,...,...,...,...,...,...,...
35454,ING6-MYST4,O95696_P55201_Q8WYB5_Q8WYH8,BRD1_BRPF1_ING5_KAT6B,0:0:0:0,SPIKE,None,None,None,None,1,1
35455,RNAPIIo,P19387_P24928_P30876,POLR2A_POLR2B_POLR2C,0:0:0,SPIKE,None,None,None,None,1,1
35456,ING5-MYST2,O95251_Q6IE81_Q8WYH8,ING5_JADE1_KAT7,0:0:0,SPIKE,None,None,None,None,1,1
35457,MCM,P25205_P33991_P33992_P33993_P49736_Q14566_Q7L590,MCM10_MCM2_MCM3_MCM4_MCM5_MCM6_MCM7,0:0:0:0:0:0:0,SPIKE,None,None,None,None,1,1


In [69]:
available_ligands = categories[categories["category"]=="ligand"]["uniprot"].tolist()
available_receptors = categories[categories["category"]=="receptor"]["uniprot"].tolist()

In [70]:
resource = resource[(resource["source"].isin(available_ligands)) & (resource["target"].isin(available_receptors))]

### map from UniProt to HGNC

In [71]:
source_mapping = pd.read_csv("/data/source_mapping.tsv",sep="\t")
dest_mapping = pd.read_csv("/data/dest_mapping.tsv",sep="\t")

In [72]:
resource = resource[(resource["source"].isin(source_mapping["From"]) & (resource["target"].isin(source_mapping["From"])))]
resource = resource[["source","target","references"]].drop_duplicates()

In [73]:
resource

,source,target,references
26,P19235,Q13507,TRIP:18276585;TRIP:19074769;TRIP:21757714
38,P06241,Q9UBN4,TRIP:16144838;TRIP:18048348;dbPTM:16144838
76,P06241,Q9HBA0,HPRD:12538589;SPIKE_LC:16713569;TRIP:12538589
94,Q9UEF7,Q9NQA5,TRIP:16239475;TRIP:18495742;TRIP:18606998
96,P60903,Q9NQA5,Lit-BM-17:12660155;TRIP:12660155
...,...,...,...
92130,P37840,Q02156,HPRD:10407019
92132,P37840,P07101,HPRD:11943812
92185,P41159,P54646,HPRD:11797013
92188,P06241,P15391,HPRD:7589101;SPIKE_LC:16713569


### reassign Uniprot IDs in Omnipath to HGNC

In [74]:
source_mapping.index = source_mapping["From"].tolist()
dest_mapping.index = dest_mapping["From"].tolist()
source_mapping  = source_mapping.drop_duplicates(["From"])
dest_mapping  = dest_mapping.drop_duplicates(["From"])

In [75]:
resource["source"] = source_mapping.loc[resource["source"].tolist()]["To"].tolist()
resource["target"] = dest_mapping.loc[resource["target"].tolist()]["To"].tolist()

In [82]:
resource.to_csv("/data/LR_database/intercell_Omnipath.csv")

In [83]:
nodes = pd.DataFrame({"category":["ligand"]*resource.shape[0] + ["receptor"]*resource.shape[0], "identifier":resource["source"].tolist() + resource["target"].tolist()})

In [85]:
nodes["Id"] = range(nodes.shape[0])

In [88]:
nodes = nodes[["Id","category","identifier"]]

In [89]:
nodes.to_csv("/data/LR_database/intercell_nodes.csv")

In [94]:
nodes.index = nodes["identifier"].tolist()

In [ ]:
resource["source"] = nodes.loc[resource["source"].tolist()]["Id"].tolist()
resource["target"] = nodes.loc[resource["target"].tolist()]["Id"].tolist()